In [1]:
import numpy as np
import scipy.linalg as la

In [2]:
A = np.array([
    [1.5, 0.5],
    [0.5, 1.5],
    ]).astype('complex')
U = la.expm(1j*A)
eigs_u, vecs_u = np.linalg.eig(U)
print(vecs_u[1])

[-0.70710678+1.73472348e-16j  0.70710678+3.33066907e-16j]


In [3]:
#states
state_zero = np.array([[1.0],[0.0]])
state_one = np.array([[0.0],[1.0]])
#projectors
P0 = np.dot(state_zero, state_zero.T)
P1 = np.dot(state_one, state_one.T)

In [4]:
def multi_kron(*args):
    ret = np.array([[1.0]]).astype('complex')
    for q in args:
        ret = np.kron(ret, q)
    return ret

def multi_dot(*args):
    ret = np.eye(np.shape(args[0])[0]).astype('complex')
    for q in args:
        ret = np.dot(ret, q)
    return ret

In [5]:
def H():
    return 1/np.sqrt(2) * np.array([[1, 1],
                                    [1, -1]]).astype('complex')
def X():
    return np.array([[0, 1],
                     [1, 0]]).astype('complex')
def Rz(phi):
    return np.array([[np.exp(-1j*phi/2), 0],
                     [0, np.exp(1j*phi/2)]]).astype('complex')
def P(phi):
    return np.array([[1, 0],
                     [0, np.exp(1j*phi)]]).astype('complex')
def Ry(theta):
    return np.array([[np.cos(theta/2), -np.sin(theta/2)],
                     [np.sin(theta/2), np.cos(theta/2)]]).astype('complex')
def I():
    return np.array([[1, 0],
                     [0, 1]]).astype('complex')

#for some reson CONTROL and TARGET are changed
def control(control_q, target_q, n_of_qubits, operator):
    list_of_ops_left = []
    list_of_ops_right = []
    for i in range(n_of_qubits):
        if i == control_q:
            list_of_ops_left.append(P0)
            list_of_ops_right.append(P1)
        elif i == target_q:
            list_of_ops_left.append(I())
            list_of_ops_right.append(operator)
        else:
            list_of_ops_left.append(I())
            list_of_ops_right.append(I())
    return multi_kron(*list_of_ops_left) + multi_kron(*list_of_ops_right).astype('complex')

In [6]:
prime_state = multi_kron(np.array([[el] for el in vecs_u[1]]), state_zero, state_zero,
                         state_zero)

In [7]:
#state preparation
state_prep = multi_kron(I(), H(), H(), H())
# prime_state = np.dot(state, prime_state)
# print(prime_state)

In [22]:
#eigenvalues to phases
etp_one = control(control_q=1,
                  target_q=0,
                  n_of_qubits=4,
                  operator=U**1) #4
etp_two = control(control_q=2,
                  target_q=0,
                  n_of_qubits=4,
                  operator=U**1) #2
etp_three = control(control_q=3,
                  target_q=0,
                  n_of_qubits=4,
                  operator=U**1) #1


eigs_to_phases = multi_dot(etp_three, etp_two, etp_two, etp_one, etp_one, etp_one, etp_one)

# prime_state = np.dot(final_u, prime_state)
# print(final_u.shape)
eigs_to_phases.round(3)
# print(U)

array([[ 1.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,
         0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,
         0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,
         0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ],
       [ 0.   +0.j   ,  0.062+0.875j,  0.   +0.j   ,  0.   +0.j   ,
         0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,
         0.   +0.j   , -0.478+0.034j,  0.   +0.j   ,  0.   +0.j   ,
         0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ],
       [ 0.   +0.j   ,  0.   +0.j   , -0.535+0.076j,  0.   +0.j   ,
         0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,
         0.   +0.j   ,  0.   +0.j   , -0.119-0.833j,  0.   +0.j   ,
         0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ],
       [ 0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   , -0.015-0.069j,
         0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,
         0.   +0.j   ,  0.   +0.j   ,  0.   +

In [10]:
from qiskit import QuantumCircuit, Aer, execute
def swap(n_of_qubs):
    """n-qubit QFTdagger the first n qubits in circ"""
    # Don't forget the Swaps!
    qc = QuantumCircuit(n_of_qubs+1)
    for qubit in range(n_of_qubs//2): #n//2
        qc.swap(qubit, n_of_qubs-qubit-1)
    backend = Aer.get_backend('unitary_simulator')
    job = execute(qc, backend)
    result = job.result()
    op = result.get_unitary(qc).data
    return op

In [11]:
#inverse_fourier
if_one = multi_kron(I(), I(), I(), H())
if_two = control(
    control_q=3,
    target_q=2,
    n_of_qubits=4,
    operator=P(-np.pi/2))

if_three = multi_kron(I(), I(), H(), I())
if_four = control(control_q=3,
                  target_q=1,
                  n_of_qubits=4,
                  operator=P(-np.pi/4))
if_five = control(control_q=2,
                  target_q=1,
                  n_of_qubits=4,
                  operator=P(-np.pi/2))
if_six = multi_kron(I(), H(), I(), I())

ift = multi_dot(swap(3), if_one, if_two, if_three, if_four, if_five, if_six)
print(ift.shape)
print(ift.round(3))

(16, 16)
[[ 0.354+0.j     0.354+0.j     0.354+0.j     0.354+0.j     0.354+0.j
   0.354+0.j     0.354+0.j     0.354+0.j     0.   +0.j     0.   +0.j
   0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j
   0.   +0.j   ]
 [ 0.354+0.j     0.25 -0.25j   0.   -0.354j -0.25 -0.25j  -0.354+0.j
  -0.25 +0.25j  -0.   +0.354j  0.25 +0.25j   0.   +0.j     0.   +0.j
   0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j
   0.   +0.j   ]
 [ 0.354+0.j     0.   -0.354j -0.354+0.j    -0.   +0.354j  0.354+0.j
   0.   -0.354j -0.354+0.j    -0.   +0.354j  0.   +0.j     0.   +0.j
   0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j
   0.   +0.j   ]
 [ 0.354+0.j    -0.25 -0.25j  -0.   +0.354j  0.25 -0.25j  -0.354+0.j
   0.25 +0.25j   0.   -0.354j -0.25 +0.25j   0.   +0.j     0.   +0.j
   0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j
   0.   +0.j   ]
 [ 0.354+0.j    -0.354+0.j     0.354+0.j    -0.354+0.j     0.354+0.j
  -0.354+0.j     0.354+0.j

In [12]:
final_u = multi_dot(ift, eigs_to_phases, state_prep)

In [17]:
# prime_state = multi_kron(state_zero, state_zero, state_zero, vecs_u[1])
final_state = np.dot(final_u, prime_state)
rho = np.dot(final_state, np.conj(final_state).T)
print(np.round(rho, 3), '\n')

[[ 0.019+0.j     0.081-0.033j -0.023+0.023j -0.006+0.014j -0.   +0.011j
   0.004+0.009j  0.007+0.007j  0.011+0.005j -0.019+0.j    -0.081+0.033j
   0.023-0.023j  0.006-0.014j -0.   -0.011j -0.004-0.009j -0.007-0.007j
  -0.011-0.005j]
 [ 0.081+0.033j  0.39 -0.j    -0.137+0.057j -0.047+0.047j -0.018+0.044j
  -0.   +0.042j  0.017+0.04j   0.038+0.038j -0.081-0.033j -0.39 +0.j
   0.137-0.057j  0.047-0.047j  0.018-0.044j -0.   -0.042j -0.017-0.04j
  -0.038-0.038j]
 [-0.023-0.023j -0.137-0.057j  0.056-0.j     0.023-0.01j   0.013-0.013j
   0.006-0.015j  0.   -0.017j -0.008-0.019j  0.023+0.023j  0.137+0.057j
  -0.056-0.j    -0.023+0.01j  -0.013+0.013j -0.006+0.015j -0.   +0.017j
   0.008+0.019j]
 [-0.006-0.014j -0.047-0.047j  0.023+0.01j   0.011+0.j     0.008-0.003j
   0.005-0.005j  0.003-0.007j -0.   -0.009j  0.006+0.014j  0.047+0.047j
  -0.023-0.01j  -0.011+0.j    -0.008+0.003j -0.005+0.005j -0.003+0.007j
   0.   +0.009j]
 [-0.   -0.011j -0.018-0.044j  0.013+0.013j  0.008+0.003j  0.006-0.j
   

In [51]:
proj = multi_kron(P0, I(), I(), P1)
prob = np.trace(np.dot(proj, rho))
print(np.round(prob, 3))
proj = multi_kron(P0, I(), I(), P0)
prob = np.trace(np.dot(proj, rho))
print(np.round(prob, 3))
proj = multi_kron(P1, I(), I(), P1)
prob = np.trace(np.dot(proj, rho))
print(np.round(prob, 3))
proj = multi_kron(P1, I(), I(), P0)
prob = np.trace(np.dot(proj, rho))
print(np.round(prob, 3))

(0.115+0j)
(0.385-0j)
(0.115-0j)
(0.385+0j)


In [86]:
proj = multi_kron(I(), P0, P0, P1)
prob = np.trace(np.dot(proj, rho))
print(np.round(prob, 4))
print(1619/2048)

(0.7803-0j)
0.79052734375


In [53]:
prob0 = np.trace(np.dot(multi_kron(I(), I(), I(), P1), rho))
prob0

(0.22984884706592987+5.302664910800047e-19j)